In [1]:
!pip install gradio transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 112.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.1 MB/s eta 0:00:00


In [2]:
import gradio as gr
from transformers import pipeline


In [23]:
qa_pipeline = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    tokenizer="deepset/roberta-base-squad2"
)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [24]:
# Longformer acepta hasta ~4000 tokens, así que dividiremos el texto si es más largo
def dividir_texto(texto, max_longitud=3900):
    oraciones = texto.split(". ")
    bloques = []
    actual = ""
    for oracion in oraciones:
        if len(actual) + len(oracion) < max_longitud:
            actual += oracion + ". "
        else:
            bloques.append(actual.strip())
            actual = oracion + ". "
    if actual:
        bloques.append(actual.strip())
    return bloques


In [25]:
def responder_largo(pregunta, archivo):
    try:
        if archivo is None:
            return "Por favor sube un archivo .txt con contenido válido."

        # Leer archivo según tipo
        if hasattr(archivo, "name"):
            with open(archivo.name, encoding="utf-8") as f:
                contexto = f.read()
        else:
            return "⚠️ No se pudo leer el archivo."

        if not pregunta.strip():
            return "Escribe una pregunta válida."

        bloques = [contexto] if len(contexto) < 3000 else dividir_texto(contexto)
        respuestas = []

        for bloque in bloques:
            resultado = qa_pipeline(question=pregunta, context=bloque, top_k=3)
            # Aseguramos que sea una lista (algunos modelos devuelven dict si top_k=1)
            if isinstance(resultado, dict):
                resultado = [resultado]
            for r in resultado:
                if r['score'] >= 0.1:  # solo respuestas confiables
                    respuestas.append((r['answer'], round(r['score'], 3)))

        if not respuestas:
            return "⚠️ No se encontró una respuesta con confianza suficiente."

        # Ordenar por score descendente y mostrar top 3
        respuestas.sort(key=lambda x: x[1], reverse=True)
        texto_respuestas = "\n".join(
            [f"• {resp} (confianza: {score})" for resp, score in respuestas[:3]]
        )
        return texto_respuestas

    except Exception as e:
        return f"⚠️ Error: {str(e)}"


In [26]:
# Interfaz Gradio
iface = gr.Interface(
    fn=responder_largo,
    inputs=[
        gr.Textbox(label="Pregunta"),
        gr.File(label="Carga un archivo .txt", file_types=[".txt"])
    ],
    outputs="text",
    title="QA con Longformer para documentos largos",
    description="Este sistema utiliza Longformer para responder preguntas en archivos largos (.txt). Divide automáticamente el contenido si supera los 4000 tokens."
)

In [27]:
iface.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4e30a57260f2d73541.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
